In [3]:
!pip install --upgrade pip


In [10]:
!pip install allennlp


In [12]:
import numpy as np
from sklearn.svm import SVC



In [14]:
!pip install allennlp-models


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.7 MB/s eta 0:00:00
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
INFO: pip is looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 33.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of cached-path to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
!pip install numpy


In [27]:
!apt install python3-numpy


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  python-numpy-doc python3-pytest python3-numpy-dbg
The following NEW packages will be installed:
  python3-numpy
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 2,724 kB of archives.
After this operation, 14.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-numpy amd64 1:1.17.4-5ubuntu3.1 [2,724 kB]
Fetched 2,724 kB in 0s (7,708 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../python3-numpy_1%3a1.17.4-5ubuntu3.1_amd64.deb ...
Unpacking python3-numpy (1:1.17.4-5ubuntu3.1) ...
Setting up python3-numpy (1:1.17.4-5ubuntu3.1) ...


In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score

In [40]:
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, MetadataField
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data import Instance, Vocabulary, Batch
from allennlp.modules.token_embedders import ElmoTokenEmbedder
from allennlp.data.tokenizers import SpacyTokenizer

In [48]:
# Specify the ELMo options and weights files
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"


In [47]:
# Create an instance of the ElmoTokenEmbedder
elmo_token_embedder = ElmoTokenEmbedder(options_file=options_file, weight_file=weight_file)

In [18]:
import pandas as pd
# Load the dataset into a pandas DataFrame
df = pd.read_csv('/content/Restaurant_Reviews.csv')  # Assuming 'reviews.csv' contains 'text' and 'liked' columns

In [35]:
# Extract text and labels from the dataset
texts = df["Review"].values
labels = df["Liked"].values

In [36]:
# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [41]:
# Create token indexers
token_indexers = {"tokens": SingleIdTokenIndexer()}

# Tokenize and create TextField for training sentences
train_instances = []
tokenizer = SpacyTokenizer(language="en_core_web_sm", pos_tags=False)
for text, label in zip(train_texts, train_labels):
    tokens = tokenizer.tokenize(text)
    token_field = TextField(tokens, token_indexers)
    fields = {"tokens": token_field, "label": MetadataField(label)}
    instance = Instance(fields)
    train_instances.append(instance)
train_batch = Batch(train_instances)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [43]:
from allennlp.modules.token_embedders import ElmoTokenEmbedder

In [69]:
from allennlp.modules.elmo import Elmo, batch_to_ids
# Create an instance of the Elmo class
elmo = Elmo(options_file, weight_file, num_output_representations=1)

In [70]:
# Tokenize the sentences and convert them to character IDs
train_character_ids = batch_to_ids(train_texts)
test_character_ids = batch_to_ids(test_texts)

In [ ]:
import torch

In [ ]:
# Embed the training data
with torch.no_grad():
    train_embeddings = elmo(train_character_ids)["elmo_representations"][0].detach().numpy()


In [ ]:
# Embed the test data
with torch.no_grad():
    test_embeddings = elmo(test_character_ids)["elmo_representations"][0].detach().numpy()

In [ ]:
# Train an SVM classifier
svm = SVC()
svm.fit(train_embeddings, train_labels)

# Predict labels for test data
predictions = svm.predict(test_embeddings)

# Calculate precision, recall, and F-score
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F-score:", f1)
